In [1]:
input_file = "input_files/day_05_1.txt"

with open(input_file) as lines:
    data = lines.read().strip()


## Helper class for interval logic

In [2]:
class Interval:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
        
    def overlaps(self, other):
        ''' returns True if any part of the intervals overlaps, otherwise False'''
        return max(self.start, other.start) < min(self.stop,other.stop)

    def __contains__(self, n):
        return n >= self.start and n < self.stop
    
    def convert_interval(self, other):
        '''
        this returns a converted interval (or none if no overlap
        and a list of the remaining partitions of the original
        '''
        other_interval, delta = other
        if not self.overlaps(other_interval):
            return None, [self]
        
        remainders = []
        
        start = max(self.start, other_interval.start)
        stop = min(self.stop, other_interval.stop)
        
        converted = Interval(start + delta, stop + delta)
        
        if start > self.start:
            remainders.append(Interval(self.start, other_interval.start))
        if stop < self.stop:
            remainders.append(Interval(other_interval.stop, self.stop))
            
        return converted, remainders
    
    def convert_interval_list(self, conversions):
        '''
        A single interval can be effected by more than one conversion interval.
        This will convert all the conversions that overlap and
        return any inconverted remainders.
        '''
        total_remaining = [self]
        total_conversions = []
        for conversion in conversions:
            conversion
            left_overs = []
            for interval in total_remaining:
                converted, remainders = interval.convert_interval(conversion)
                if converted:
                    total_conversions.append(converted)
                left_overs.extend(remainders)
            total_remaining = left_overs
        return total_conversions, total_remaining     
        
    def __repr__(self):
        return f'Interval({self.start}, {self.stop})'
    
i1 = Interval(10, 20)
i2 = (Interval(11, 15), 100)
i3 = (Interval(15, 17), 100)

i1.convert_interval_list([i2, i3])

([Interval(111, 115), Interval(115, 117)],
 [Interval(10, 11), Interval(17, 20)])

## Parse Data

In [3]:
def make_conversion(lines):
    '''
    convert the lines of a particular conversion to 
    Interval and delta tuples
    '''
    ranges = []
    for line in lines:
        dest, source, n = map(int, line.split())
        ranges.append((Interval(source, source+n), dest - source))
    return ranges

def parse_data(data):
    '''
    return raw seed numbers and list of list of range tuples
    '''
    categories = data.split('\n\n')
    seeds = [int(n) for n in categories[0][categories[0].index(":")+1:].split()]
    conversions = [make_conversion(cat.split('\n')[1: ]) for cat in categories[1:]]
    return seeds, conversions

seeds, conversions = parse_data(data)


## Part One

In [4]:
def get_conversion(seed, conversions):
    '''
    run a seed through a single conversion set
    '''
    for conversion in conversions:
        conversions_interval, delta = conversion
        if seed in conversions_interval:
            return seed + delta
    return seed

def get_location(seed, conversions):
    '''run a single seed through all conversions'''
    for c in conversions:
        seed = get_conversion(seed, c)
    return seed

min(get_location(s, conversions) for s in seeds)

1181555926

# Part 2

For each conversion, convert all the intervals. At first the intervals are just the seeds
but as conversions partially effect the seed intervals, the number of intervals increases.

At the end the interval with the lowest start contains the minimum location in the `start` property.


In [5]:
# seeds are now ranges
total_ranges = [Interval(start, start+length) for start, length in  zip(seeds[::2], seeds[1::2])]

for conversion in conversions:
    converted = []
    left_overs = []
    for seed_range in total_ranges:
        loval_converted, local_leftovers = seed_range.convert_interval_list(conversion)
        converted.extend(loval_converted)
        left_overs.extend(local_leftovers)
    total_ranges = converted + left_overs

min_interval = min(total_ranges, key=lambda r: r.start)
min_interval.start

37806486